In [1]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

import jupytext
import otter
from ipywidgets import interact

grader = otter.Notebook("main.ipynb")
%reload_ext divewidgets

# Lab 7: Cybersecurity

**CS1302 Introduction to Computer Programming**
___

Python is a popular tool among hackers and engineers. In this lab, you will learn Cryptology in cybersecurity, which covers
- [Cryptography](https://en.wikipedia.org/wiki/Cryptography): Encryption and decryption using a cipher.
- [Cryptanalysis](https://en.wikipedia.org/wiki/Cryptanalysis): Devising an attack to break a cipher.

## Caesar symmetric key cipher

We first implement a simple cipher called the [Caesar cipher](https://en.wikipedia.org/wiki/Caesar_cipher).

In [2]:
%%html
<iframe width="800" height="415" src="https://www.youtube.com/embed/sMOZf4GN3oc" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### Encrypt/decrypt a character

**How to encrypt a character?**

The following code encrypts a character `char` using a non-negative integer `key`.

In [3]:
cc_n = 1114112


def cc_encrypt_character(char, key):
    """
    Return the encryption of a character by an integer key using Caesar cipher.

    Parameters
    ----------
    char: str
        a unicode (UTF-8) character to be encrypted.
    key int:
        secret key to encrypt char.
    """
    char_code = ord(char)
    shifted_char_code = (char_code + key) % cc_n
    encrypted_char = chr(shifted_char_code)
    return encrypted_char

For example, to encrypt the letter `'A'` using a secret key `5`:

In [4]:
cc_encrypt_character("A", 5)

'F'

The character `'A'` is encrypted to the character `'F'` as follows:

1. `ord(char)` return the integer `65`, which is the code point (integer representation) of the unicode of `'A'`. 
2. `(char_code + key) % cc_n` cyclic shifts the code by the key `5`.
3. `chr(shifted_char_code)` converts the shifted code back to a character, which is `'F'`.

| Encryption                      |     |       |     |     |     |     |     |     |
| ------------------------------- | --- | ----- | --- | --- | --- | --- | --- | --- |
| `char`                          | ... | **A** | B   | C   | D   | E   | F   | ... |
| `ord(char)`                     | ... | **65**| 66  | 67  | 68  | 69  | 70  | ... |
| `(ord(char) + key) % cc_n`      | ... | **70**| 71  | 72  | 73  | 74  | 75  | ... |
| `(chr(ord(char) + key) % cc_n)` | ... | **F** | G   | H   | I   | J   | K   | ... |

You may learn more about `ord` and `chr` from their docstrings:

In [5]:
help(ord)
help(chr)

Help on built-in function ord in module builtins:

ord(c, /)
    Return the Unicode code point for a one-character string.

Help on built-in function chr in module builtins:

chr(i, /)
    Return a Unicode string of one character with ordinal i; 0 <= i <= 0x10ffff.



**How to decrypt a character?**

Mathematically, we define the encryption and decryption of a character for Caesar cipher as

$$ \begin{aligned} E(x,k) &:= x + k \mod n & \text{(encryption)} \\
D(x,k) &:= x - k \mod n & \text{(decryption),} \end{aligned}
$$
where $x$ is the character code in $\{0,\dots,n\}$ and $k$ is the secret key. `mod` operator above is the modulo operator. In Mathematics, it has a lower precedence than addition and multiplication and is typeset with an extra space accordingly.

The encryption and decryption satisfy the recoverability condition

$$ D(E(x,k),k) = x $$
so two people with a common secret key can encrypt and decrypt a character, but others without the key cannot. This defines a [symmetric cipher](https://en.wikipedia.org/wiki/Symmetric-key_algorithm).

The following code decrypts a character using a key.

In [6]:
def cc_decrypt_character(char, key):
    """
    Return the decryption of a character by the key using Caesar cipher.

    Parameters
    ----------
    char: str
        a unicode (UTF-8) character to be decrypted.
    key: int
        secret key to decrypt char.
    """
    char_code = ord(char)
    shifted_char_code = (char_code - key) % cc_n
    decrypted_char = chr(shifted_char_code)
    return decrypted_char

For instance, to decrypt the letter `'F'` by the secret key `5`:

In [7]:
cc_decrypt_character("F", 5)

'A'

The character `'F'` is decrypted back to `'A'` because
`(char_code - key) % cc_n` reverse cyclic shifts the code by the key `5`.

| Encryption                      |     |       |     |     |     |     |     |     | Decryption                      |
| ------------------------------- | --- | ----- | --- | --- | --- | --- | --- | --- | ------------------------------- |
| `char`                          | ... | **A** | B   | C   | D   | E   | F   | ... | `(chr(ord(char) - key) % cc_n)` |
| `ord(char)`                     | ... | **65**| 66  | 67  | 68  | 69  | 70  | ... | `(ord(char) - key) % cc_n`      |
| `(ord(char) + key) % cc_n`      | ... | **70**| 71  | 72  | 73  | 74  | 75  | ... | `ord(char)`                     |
| `(chr(ord(char) + key) % cc_n)` | ... | **F** | G   | H   | I   | J   | K   | ... | `char`                          |

**Exercise** (modulo-arithmetic)

Why did we set `cc_n = 1114112`? Explain whether the recoverability property may fail if we set `cc_n` to a bigger number or remove `% cc_n` for both `cc_encrypt_character` and `cc_decrypt_character`.

**Answer**

(1)

`help(chr)` reads that in `chr(i), 0 <= i <= 0x10ffff` (`0x10ffff=1114111`), so `1114112` is the total possible number of characters in UTF-8, which is also known as the total size of Unicode code space.

In `cc_encrypt_character` and `cc_decrypt_character`, the character as the argument is in UTF-8. So the returned character should also be in UTF-8.

(2)

When calling `chr(i)` with `i` out of that range, Python will raise a `ValueError` saying that `chr() arg not in range(0x110000)`.

* If `cc_n` is set to a larger number,
    * `cc_encrypt_character` can return a character whose number is larger than 0x110000, which might raise the error.
    * If the encrypted charcater corresponds to a very small number, `cc_decrypt_character` can return a character whose number is larger than the encrypted character (`-x % cc_n = cc_n - x`, as `cc_n` becomes larger, `cc_n - x` also becomes larger).
* If `% cc_n` is removed,
    * `cc_encrypt_character` can return a character whose number is larger than 0x110000, which might raise the error.
    * `cc_decrypt_character` can return a character whose number issmaller than 0, which might raise the error.

So the recoverability property might fail in both cases.


### Encrypt a plaintext and decrypt a ciphertext

Of course, it is more interesting to encrypt a string instead of a character. The following code implements this in one line.

In [8]:
def cc_encrypt(plaintext, key):
    """
    Return the ciphertext of a plaintext by the key using the Caesar cipher.

    Parameters
    ----------
    plaintext: str
        A unicode (UTF-8) message to be encrypted.
    public_key: int
        Public key to encrypt plaintext.
    """
    return "".join([chr((ord(char) + key) % cc_n) for char in plaintext])

The above function encrypts a message, referred to as the *plaintext*, by replacing each character with its encryption.  
This is referred to as a [*substitution cipher*](https://en.wikipedia.org/wiki/Substitution_cipher).

**Exercise** (cc-decrypt)

Define a function `cc_decrypt` that
- takes a string `ciphertext` and an integer `key`, and
- returns the plaintext that encrypts to `ciphertext` by the key using Caesar cipher.

In [9]:
def cc_decrypt(ciphertext, key):
    """
    Return the plaintext that encrypts to ciphertext by the key using Caesar cipher.

    Parameters
    ----------
    ciphertext: str
        message to be decrypted.
    key: int
        secret key to decrypt the ciphertext.
    """
    return "".join([chr((ord(char) - key) % cc_n) for char in ciphertext])

In [10]:
grader.check("cc-decrypt")

cc-decrypt results: All test cases passed!

## Brute-force attack

### Create an English dictionary

You will launch a brute-force attack to guess the key that encrypts an English text. The idea is simple: 

- You try decrypting the ciphertext with different keys, and 
- see which of the resulting plaintexts make the most sense (most English-like).

To check whether a plaintext is English-like, we need to have a list of English words. One way is to type them out
but this is tedious. Alternatively, we can obtain the list from the *Natural Language Toolkit (NLTK)*:

In [11]:
!pip install nltk

In [12]:
import nltk

nltk.download("words")
from nltk.corpus import words

[nltk_data] Downloading package words to /home/fwang247/nltk_data...
[nltk_data]   Package words is already up-to-date!


`words.words()` returns a list of words. We can check whether a string is in the list using the operator `in`.

In [13]:
for word in "Ada", "ada", "Hello", "hello":
    print("{!r} in dictionary? {}".format(word, word in words.words()))

'Ada' in dictionary? True
'ada' in dictionary? False
'Hello' in dictionary? False
'hello' in dictionary? True


However, there are two issues:
- Checking membership is slow for a long list.
- Both 'Hello' and 'ada' are English-like but not in the words list.

**Exercise** (dictionary)

Using the method `lower` of `str` and the constructor `set`, assign `dictionary` to a set of lowercase English words from `words.words()`.

In [14]:
dictionary = set(word.lower() for word in words.words())

In [15]:
grader.check("dictionary")

dictionary results: All test cases passed!

### Identify English-like text

To determine how English-like a text is, we calculate the following score:

$$
\frac{\text{number of English words in the text}}{\text{number of tokens in the text}} 
$$
where tokens are substrings, not necessarily English words, separated by white space characters in the text.

In [16]:
def tokenizer(text):
    """Returns the list of tokens of the text."""
    return text.split()


def get_score(text):
    """Returns the fraction of tokens which appear in dictionary."""
    tokens = tokenizer(text)
    words = [token for token in tokens if token in dictionary]
    return len(words) / len(tokens)


# tests
get_score("hello world"), get_score("Hello, World!")

(1.0, 0.0)

As shown in the tests above, the code fails to handle text with punctuations and uppercase letters properly.  
In particular, 
- while `get_score` recognizes `hello world` as English-like and returns the maximum score of 1, 
- it fails to recognize `Hello, World!` as English-like and returns the minimum score of 0.

Why? Every word in `dictionary`
- are in lowercase, and
- have no leading/trailing punctuations.

**Exercise** (tokenizer)

Define a function `tokenizer` that 
- takes a string `text` as an argument, and
- returns a `list` of tokens obtained by
  1. splitting `text` into a list using `split()`;
  2. removing leading/trailing punctuations in `string.punctuation` using the `strip` method; and
  3. converting all items of the list to lowercase using `lower()`.

In [17]:
import string


def tokenizer(text):
    """Returns the list of tokens of the text such that
    1) each token has no leading or trailing spaces/punctuations, and
    2) all letters in each token are in lowercase."""
    tokens = text.split()
    return [token.strip(string.punctuation).lower() for token in tokens]

In [18]:
grader.check("tokenizer")

tokenizer results: All test cases passed!

### Launch a brute-force attack

**Exercise** (cc-attack)

Define the function `cc_attack` that 
- takes as arguments
    - a string `ciphertext`,
    - a floating point number `threshold` in the interval $(0,1)$ with a default value of $0.6$, and
- returns a generator that  
    - generates one-by-one in ascending order guesses of the key that
    - decrypt `ciphertext` to texts with scores at least the `threshold`.

In [19]:
def cc_attack(ciphertext, threshold=0.6):
    """Returns a generator that generates the next guess of the key that
    decrypts the ciphertext to a text with get_score(text) at least the threshold.
    """
    for i in range(cc_n):
        if get_score(cc_decrypt(ciphertext, i)) >= threshold:
            yield i

In [20]:
grader.check("cc-attack")

cc-attack results: All test cases passed!

## Challenge

Another symmetric key cipher is [columnar transposition cipher](https://en.wikipedia.org/wiki/Transposition_cipher#Columnar_transposition). A transposition cipher encrypts a text by permuting instead of substituting characters.

**Exercise** (columnar)

Study and implement the irregular case of the [columnar transposition cipher](https://en.wikipedia.org/wiki/Transposition_cipher#Columnar_transposition) as described in the Wikipedia page. Define the functions 
- `ct_encrypt(plaintext, key)` for encryption, and 
- `ct_decrypt(ciphertext, key)` for decryption. 

You can assume the plaintext is in uppercase and has no spaces/punctuations.

```{hint}
See the test cases for examples of `plaintext`, `key`, and the corresponding `ciphertext`.
```

In [21]:
'''
argsort('ZEBRAS') = (4, 2, 1, 3, 5, 0)
'''


def argsort(seq):
    '''A helper function that returns the tuple of indices that would sort the
    sequence seq.'''
    return tuple(x[0] for x in sorted(enumerate(seq), key=lambda x: x[1]))

'''
ct_idx(25, 'ZEBRAS') = 
[4, 10, 16, 22, 2, 8, 14, 20, 1, 7, 13, 19, 3, 9, 15, 21, 5, 11, 17, 23, 0, 6, 12, 18, 24]
'''


def ct_idx(length, key):
    '''A helper function that returns the tuple of indices that would permute 
    the letters of a message according to the key using the irregular case of 
    columnar transposition cipher.'''
    # seq = tuple(range(length))
    # return [i for j in argsort(key) for i in ...]
    cols = len(key)
    lst = []
    for number_col, write_col in enumerate(argsort(key)):
        lst.extend(range(write_col, length, cols))
    return lst



'''
inv_ct_idx(25, 'ZEBRAS') = [
    20, 21, 22, 23, 24, # 0
    8, 9, 10, 11,       # 1
    4, 5, 6, 7,         # 2
    12, 13, 14, 15,     # 3
    0, 1, 2, 3,         # 4
    16, 17, 18, 19,     # 5
]
'''


def inv_ct_idx(length, key):
    ans = [0 for _ in range(length)]
    for i, j in enumerate(ct_idx(length, key)):
        ans[j] = i
    return ans


def ct_encrypt(plaintext, key):
    '''
    Return the ciphertext of a plaintext by the key using the irregular case
    of columnar transposition cipher.

    Parameters
    ----------
    plaintext: str
        a message in uppercase without punctuations/spaces.
    key: str
        secret key to encrypt plaintext.
    '''
    return ''.join([plaintext[i] for i in ct_idx(len(plaintext), key)])


def ct_decrypt(ciphertext, key):
    '''
    Return the plaintext of the ciphertext by the key using the irregular case
    of columnar transposition cipher.

    Parameters
    ----------
    ciphertext: str
        a string in uppercase without punctuations/spaces.
    key: str
        secret key to decrypt ciphertext.
    '''
    return ''.join([ciphertext[i] for i in inv_ct_idx(len(ciphertext), key)])

In [22]:
grader.check("columnar")

columnar results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [23]:
# extra files to submit
extra_files = []

In [24]:
# Generate the source main.py necessary for grading and similarity check.
jupytext.write(jupytext.read("main.ipynb"), "main.py", fmt="py:percent")

In [25]:
# Generate the zip file to submit.
grader.export(pdf=False, run_tests=False, files=["main.py",  *extra_files])